<a href="https://colab.research.google.com/github/arthurpeter/Machine_Learning/blob/main/Lab_03_CNN_Architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Architectures

#### Motivation
In the previous Lab we saw that it is difficult to construct a custom CNN for specific tasks as we have to experiment with many hyperparameters (number of layers, number of feature maps per layer, kernel sizes, fully connected network configuration, etc.)

Luckily we do not have to do that. There are plenty of architectures that have been constructed and tested for many tasks.

With very minor changes, we can adapt existing architectures to our task and be relatively confident that it will obtain good results (if trained correctly).

In Lab 3 you will learn how to utilize available architectures and how to fine-tune them.

#### Notes
As we utilize deep networks, training will take quite a long time in this lab. Running on GPU is necessary (Runtime -> Change Runtime Type).

If you see that a run is not going anywhere (Validation Accuracy is not increasing or Loss is not decreasing) cancel it to save some time.

If possible, run the training with higher batch sizes to speed up the training.

While the models are training, we recommend that you look over the papers of the architectures.😄

#### Libraries
There are plenty of architectures available in the [torchvision library](https://pytorch.org/vision/0.8/models.html), the [timm library](https://timm.fast.ai/), and on [Hugging Face](https://huggingface.co/models?pipeline_tag=image-classification&sort=downloads).

### Training an off-the-shelf model

In [1]:
import sys

import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader

We will use the CIFAR10 dataset as in the previous lab. Let's see if we can obtain better results this time.

In [2]:
train_transform = transforms.Compose(
    [
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)

# The batch size is the number of images the model processes in parallel
# We use shuffle for training as we don't want the model to see the images in the same order
trainloader = DataLoader(trainset, batch_size=256,
                                          shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)

# For testing we don't have to shuffle the data
testloader = DataLoader(testset, batch_size=256,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:04<00:00, 36.4MB/s]


Code for training and validation - same as in previous lab

You should already be familiar with the following blocks of code.

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def train_epoch(model, dataloader, device, optimizer, criterion, epoch):
    # We set the model to be in training mode
    model.train()

    total_train_loss = 0.0
    dataset_size = 0

    # This is only for showing the progress bar
    bar = tqdm(enumerate(dataloader), total=len(dataloader), colour='cyan', file=sys.stdout)

    # We iterate through all batches - 1 step is 1 batch of batch_size images
    for step, (images, labels) in bar:
        # We take the images and their labels and push them on GPU
        images = images.to(device)
        labels = labels.to(device)

        batch_size = images.shape[0]

        # Reset gradients
        optimizer.zero_grad()

        # Obtain predictions
        pred = model(images)

        # Compute loss for this batch
        loss = criterion(pred, labels)

        # Compute gradients for each weight (backpropagation)
        loss.backward()

        # Update weights based on gradients (gradient descent)
        optimizer.step()

        # We keep track of the average training loss
        total_train_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = np.round(total_train_loss / dataset_size, 2)
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss)

    return epoch_loss

In [5]:
def valid_epoch(model, dataloader, device, criterion, epoch):
    # We set the model in evaluation mode
    model.eval()

    total_val_loss = 0.0
    dataset_size = 0

    # We keep track of correct predictions
    correct = 0

    # This is only for showing the progress bar
    bar = tqdm(enumerate(dataloader), total=len(dataloader), colour='cyan', file=sys.stdout)

    for step, (images, labels) in bar:
        images = images.to(device)
        labels = labels.to(device)

        batch_size = images.shape[0]

        pred = model(images)
        loss = criterion(pred, labels)

        # The raw output of the model is a score for each class
        # We keep the index of the class with the highest score as the prediction
        _, predicted = torch.max(pred, 1)

        # We see how many predictions match the ground truth labels
        correct += (predicted == labels).sum().item()

        # We compute evaluation metrics - loss and accurarcy
        total_val_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = np.round(total_val_loss / dataset_size, 2)

        accuracy = np.round(100 * correct / dataset_size, 2)

        bar.set_postfix(Epoch=epoch, Valid_Acc=accuracy, Valid_Loss=epoch_loss)

    return accuracy, epoch_loss

In [6]:
def run_training(model, num_epochs, learning_rate):
    # Define criterion
    criterion = nn.CrossEntropyLoss()

    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Check if we are using GPU
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    # For keeping track of the best validation accuracy
    top_accuracy = 0.0

    # We train the emodel for a number of epochs
    for epoch in range(num_epochs):

        train_loss = train_epoch(model, trainloader, device, optimizer, criterion, epoch)

        # For validation we do not keep track of gradients
        with torch.no_grad():
            val_accuracy, val_loss = valid_epoch(model, testloader, device, criterion, epoch)
            if val_accuracy > top_accuracy:
                print(f"Validation Accuracy Improved ({top_accuracy} ---> {val_accuracy})")
                top_accuracy = val_accuracy
        print()

Code for counting number of parameters of a model.

As we have limited compute resources, we will opt for models with fewer parameters.

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### SqueezeNet from torchvision

First, let's experiment with a model from the torchvision library

In [8]:
import torchvision.models as models

Let's see what models are available

In [9]:
models.list_models()

['alexnet',
 'convnext_base',
 'convnext_large',
 'convnext_small',
 'convnext_tiny',
 'deeplabv3_mobilenet_v3_large',
 'deeplabv3_resnet101',
 'deeplabv3_resnet50',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b2',
 'efficientnet_b3',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_v2_l',
 'efficientnet_v2_m',
 'efficientnet_v2_s',
 'fasterrcnn_mobilenet_v3_large_320_fpn',
 'fasterrcnn_mobilenet_v3_large_fpn',
 'fasterrcnn_resnet50_fpn',
 'fasterrcnn_resnet50_fpn_v2',
 'fcn_resnet101',
 'fcn_resnet50',
 'fcos_resnet50_fpn',
 'googlenet',
 'inception_v3',
 'keypointrcnn_resnet50_fpn',
 'lraspp_mobilenet_v3_large',
 'maskrcnn_resnet50_fpn',
 'maskrcnn_resnet50_fpn_v2',
 'maxvit_t',
 'mc3_18',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet_v2',
 'mobilenet_v3_large',
 'mobilenet_v3_small',
 'mvit_v1_b',
 'mvit_v2_s',
 'quantized_googlenet',
 '

We will choose SqueezeNet which is a very lightweight model. More details about the architecture can be found in this [paper](https://arxiv.org/abs/1602.07360).

<img src=https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-26_at_6.04.32_PM.png width=750px>

In [10]:
squeeze_net = models.squeezenet1_0()
print(squeeze_net)
print("Number of parameters:", count_parameters(squeeze_net))

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

Visualization of modifying prediction head. In our case, we only change the final output layer.

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/0*8Z3To8OAwBBIj66p.jpg" width=700px>

The classifier is built for 1000 classes as it was intended for training on ImageNet. We have to modify it to work on 10 classes. Notice that the number of parameters changes

In [11]:
squeeze_net.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1,1), stride=(1,1))
squeeze_net.to(device)
print("Number of parameters:", count_parameters(squeeze_net))

Number of parameters: 740554


We run the training

In [12]:
# You may want to change the hyperparameters to obtain better results
run_training(model=squeeze_net, num_epochs=10, learning_rate=0.001)

[INFO] Using GPU: Tesla T4

100%|██████████| 40/40 [00:02<00:00, 14.72it/s, Epoch=0, Valid_Acc=28.4, Valid_Loss=1.99]
Validation Accuracy Improved (0.0 ---> 28.36)

100%|██████████| 40/40 [00:03<00:00, 13.08it/s, Epoch=1, Valid_Acc=28.1, Valid_Loss=1.93]

100%|██████████| 40/40 [00:02<00:00, 16.10it/s, Epoch=2, Valid_Acc=30, Valid_Loss=1.95]
Validation Accuracy Improved (28.36 ---> 30.02)

100%|██████████| 40/40 [00:02<00:00, 16.33it/s, Epoch=3, Valid_Acc=35.6, Valid_Loss=1.86]
Validation Accuracy Improved (30.02 ---> 35.65)

100%|██████████| 40/40 [00:02<00:00, 15.36it/s, Epoch=4, Valid_Acc=38.8, Valid_Loss=1.76]
Validation Accuracy Improved (35.65 ---> 38.82)

100%|██████████| 40/40 [00:03<00:00, 12.99it/s, Epoch=5, Valid_Acc=39.8, Valid_Loss=1.71]
Validation Accuracy Improved (38.82 ---> 39.83)

100%|██████████| 40/40 [00:02<00:00, 15.89it/s, Epoch=6, Valid_Acc=40.2, Valid_Loss=1.73]
Validation Accuracy Improved (39.83 ---> 40.16)

100%|██████████| 40/40 [00:02<00:00, 15.96it/s, Epo

### Fine-tuning pre-trained model

The performance is not that impressive. You could improve the results by playing with the learning rate, number of epochs, and add image augmentations.

However, we do not have to start from randomly initialized weights.

We can start from weights pre-trained on large datasets (ImageNet) and fine-tune the weights on the new task.

This is simply done by specifying :```pretrained=True``` when initializing the model

This should improve the performance as the convolutional layers have already learnt to extract meaningful features.

In [13]:
squeeze_net = models.squeezenet1_0(pretrained=True)
squeeze_net.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1,1), stride=(1,1))
squeeze_net.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-b66bff10.pth


100%|██████████| 4.78M/4.78M [00:00<00:00, 78.1MB/s]


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [14]:
# Usually when we fine-tune pretrained models we utilize lower learning rates
run_training(model=squeeze_net, num_epochs=10, learning_rate=0.0003)

[INFO] Using GPU: Tesla T4

100%|██████████| 40/40 [00:02<00:00, 15.61it/s, Epoch=0, Valid_Acc=54.5, Valid_Loss=1.3]
Validation Accuracy Improved (0.0 ---> 54.52)

100%|██████████| 40/40 [00:02<00:00, 15.75it/s, Epoch=1, Valid_Acc=67.3, Valid_Loss=0.96]
Validation Accuracy Improved (54.52 ---> 67.27)

100%|██████████| 40/40 [00:02<00:00, 15.85it/s, Epoch=2, Valid_Acc=70.3, Valid_Loss=0.86]
Validation Accuracy Improved (67.27 ---> 70.31)

100%|██████████| 40/40 [00:02<00:00, 14.51it/s, Epoch=3, Valid_Acc=71.3, Valid_Loss=0.84]
Validation Accuracy Improved (70.31 ---> 71.31)

100%|██████████| 40/40 [00:02<00:00, 15.98it/s, Epoch=4, Valid_Acc=74.4, Valid_Loss=0.76]
Validation Accuracy Improved (71.31 ---> 74.43)

100%|██████████| 40/40 [00:02<00:00, 16.28it/s, Epoch=5, Valid_Acc=74.7, Valid_Loss=0.74]
Validation Accuracy Improved (74.43 ---> 74.7)

100%|██████████| 40/40 [00:02<00:00, 13.42it/s, Epoch=6, Valid_Acc=75.4, Valid_Loss=0.73]
Validation Accuracy Improved (74.7 ---> 75.41)

100%

The model should obtain better results.

### Resnet-18 from timm library

Let's try another model from a different library. We will use the timm library

In [15]:
!pip install timm

In [16]:
import timm

Let's see available models

In [17]:
timm.list_models()

['aimv2_1b_patch14_224',
 'aimv2_1b_patch14_336',
 'aimv2_1b_patch14_448',
 'aimv2_3b_patch14_224',
 'aimv2_3b_patch14_336',
 'aimv2_3b_patch14_448',
 'aimv2_huge_patch14_224',
 'aimv2_huge_patch14_336',
 'aimv2_huge_patch14_448',
 'aimv2_large_patch14_224',
 'aimv2_large_patch14_336',
 'aimv2_large_patch14_448',
 'bat_resnext26ts',
 'beit3_base_patch16_224',
 'beit3_giant_patch14_224',
 'beit3_giant_patch14_336',
 'beit3_large_patch16_224',
 'beit_base_patch16_224',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_large_patch16_224',
 'botnet26t_256',
 'botnet50ts_256',
 'caformer_b36',
 'caformer_m36',
 'caformer_s18',
 'caformer_s36',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_medium',
 'coat_lite_medium_384',
 'coat_lite_mini',
 'coat_lite_sma

We will train a ResNet-18 that is already pre-trained on ImageNet.
You can read more about ResNet in this [paper](https://arxiv.org/abs/1512.03385).

<img src=https://www.researchgate.net/publication/366608244/figure/fig1/AS:11431281109643320@1672145338540/Structure-of-the-Resnet-18-Model.jpg>

In [18]:
resnet = timm.create_model('resnet18', pretrained=True)
print(resnet)
print("Number of parameters:", count_parameters(resnet))

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

Similarly, we will need to modify the classifier to have only 10 classes.

In [19]:
resnet.fc = nn.Linear(in_features=512, out_features=10, bias=True)
resnet.to(device)
print(resnet)
print("Number of parameters:", count_parameters(resnet))

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

We run the training

In [20]:
# You may have to play with the hyperparameters to obtain better results
run_training(model=resnet, num_epochs=10, learning_rate=0.0005)

[INFO] Using GPU: Tesla T4

100%|██████████| 40/40 [00:02<00:00, 15.04it/s, Epoch=0, Valid_Acc=69.2, Valid_Loss=0.91]
Validation Accuracy Improved (0.0 ---> 69.2)

100%|██████████| 40/40 [00:02<00:00, 14.99it/s, Epoch=1, Valid_Acc=74.8, Valid_Loss=0.73]
Validation Accuracy Improved (69.2 ---> 74.79)

100%|██████████| 40/40 [00:02<00:00, 15.11it/s, Epoch=2, Valid_Acc=77.7, Valid_Loss=0.67]
Validation Accuracy Improved (74.79 ---> 77.71)

100%|██████████| 40/40 [00:02<00:00, 15.29it/s, Epoch=3, Valid_Acc=78.1, Valid_Loss=0.67]
Validation Accuracy Improved (77.71 ---> 78.12)

100%|██████████| 40/40 [00:02<00:00, 15.14it/s, Epoch=4, Valid_Acc=78.9, Valid_Loss=0.68]
Validation Accuracy Improved (78.12 ---> 78.91)

100%|██████████| 40/40 [00:03<00:00, 12.09it/s, Epoch=5, Valid_Acc=79.3, Valid_Loss=0.71]
Validation Accuracy Improved (78.91 ---> 79.33)

100%|██████████| 40/40 [00:02<00:00, 13.98it/s, Epoch=6, Valid_Acc=79.4, Valid_Loss=0.78]
Validation Accuracy Improved (79.33 ---> 79.44)

100

### MobileNet from HuggingFace

Finally, we will use a model from the Hugging Face library.

We will use the MobileNet architecture. You can read more about it in this [paper](https://arxiv.org/abs/1704.04861)

In [21]:
from transformers import AutoModelForImageClassification

We utilize `AutoModelForImageClassification.from_pretrained`to create the model.
As in the other cases, we change the classifier to have only 10 classes.

We build a wrapper around the model to only output the score for each class (by default it also outputs the loss). We only do this so it works with our train and valid functions.

In [22]:
class MobileNetWrapper(nn.Module):
  def __init__(self):
    super(MobileNetWrapper, self).__init__()
    self.model = AutoModelForImageClassification.from_pretrained("google/mobilenet_v2_1.0_224")

    # Changing the classifier to output only 10 classes
    self.model.classifier = nn.Linear(in_features=1280, out_features=10, bias=True)

  def forward(self, x):
    return self.model(x).logits

mobilenet = MobileNetWrapper()
mobilenet.to(device)
print("Number of parameters:", count_parameters(mobilenet))
print(mobilenet)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Number of parameters: 2236682
MobileNetWrapper(
  (model): MobileNetV2ForImageClassification(
    (mobilenet_v2): MobileNetV2Model(
      (conv_stem): MobileNetV2Stem(
        (first_conv): MobileNetV2ConvLayer(
          (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
          (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
          (activation): ReLU6()
        )
        (conv_3x3): MobileNetV2ConvLayer(
          (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
          (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
          (activation): ReLU6()
        )
        (reduce_1x1): MobileNetV2ConvLayer(
          (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        )
      )
  

In [23]:
# You may have to play with the hyperparameters to obtain better results
run_training(model=mobilenet, num_epochs=10, learning_rate=0.0005)

[INFO] Using GPU: Tesla T4

100%|██████████| 40/40 [00:02<00:00, 14.47it/s, Epoch=0, Valid_Acc=67.7, Valid_Loss=0.96]
Validation Accuracy Improved (0.0 ---> 67.7)

100%|██████████| 40/40 [00:02<00:00, 14.28it/s, Epoch=1, Valid_Acc=72.4, Valid_Loss=0.82]
Validation Accuracy Improved (67.7 ---> 72.37)

100%|██████████| 40/40 [00:02<00:00, 14.51it/s, Epoch=2, Valid_Acc=73.1, Valid_Loss=0.83]
Validation Accuracy Improved (72.37 ---> 73.14)

100%|██████████| 40/40 [00:02<00:00, 13.82it/s, Epoch=3, Valid_Acc=77, Valid_Loss=0.72]
Validation Accuracy Improved (73.14 ---> 77.04)

100%|██████████| 40/40 [00:03<00:00, 12.47it/s, Epoch=4, Valid_Acc=77.5, Valid_Loss=0.72]
Validation Accuracy Improved (77.04 ---> 77.54)

100%|██████████| 40/40 [00:03<00:00, 12.35it/s, Epoch=5, Valid_Acc=77.4, Valid_Loss=0.81]

100%|██████████| 40/40 [00:03<00:00, 11.92it/s, Epoch=6, Valid_Acc=77.1, Valid_Loss=0.81]

100%|██████████| 40/40 [00:03<00:00, 11.81it/s, Epoch=7, Valid_Acc=78, Valid_Loss=0.87]
Validation Ac

### Training only a subset of layers

When using pre-trained models that were trained on similar images, we do not have to re-train every layer of the network.

Ideally the first layers of the network already extract meaningful features which can be utilized in our task. So, in theory we don't have to re-train these first layers.

We will demonstrate this by fine-tuning only a part of the resnet-18 model. Let's have a closer look at the structure of ResNet18.

In [24]:
resnet = timm.create_model('resnet18', pretrained=True)

# Change the output layer to match the number of classes in CIFAR-10
resnet.fc = nn.Linear(in_features=512, out_features=10, bias=True)

resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

<img src=https://www.researchgate.net/publication/366608244/figure/fig1/AS:11431281109643320@1672145338540/Structure-of-the-Resnet-18-Model.jpg>

The model has an initial 7x7 Conv, followed by a Pooling Layer.

Afterwards, it has 4 layers, each with the same structure. Finally, it has a fully connected classifier (which we changed to output only 10 scores instead of 1000).

A layer is composed of 2 Building Blocks. A building Block contains the following:
1. A 3x3 Conv
2. Batch Norm - read more about this operation [here](https://arxiv.org/abs/1502.03167)
3. Activation - ReLU
4. A second 3x3 Conv
5. Batch Norm
6. Activation - ReLU
7. Residual Connection with the Input - input + output - read more about this [here](https://arxiv.org/abs/1512.03385)

A Building Block with residual connection looks like this:

<img src=https://miro.medium.com/v2/resize:fit:570/1*D0F3UitQ2l5Q0Ak-tjEdJg.png width=300px>

We will start by freezing all the parameters of the model. This means that we tell pytorch not to compute gradients for these parameters.

Any parameter that is frozen will not be changed during training.

In [25]:
for param in resnet.parameters():
    param.requires_grad = False

Function that prints the name of the parameter and if it's trainable or frozen.

In [26]:
def print_params(model):
    for name, param in model.named_parameters():
        print(name, param.requires_grad)

In [27]:
print_params(resnet)

conv1.weight False
bn1.weight False
bn1.bias False
layer1.0.conv1.weight False
layer1.0.bn1.weight False
layer1.0.bn1.bias False
layer1.0.conv2.weight False
layer1.0.bn2.weight False
layer1.0.bn2.bias False
layer1.1.conv1.weight False
layer1.1.bn1.weight False
layer1.1.bn1.bias False
layer1.1.conv2.weight False
layer1.1.bn2.weight False
layer1.1.bn2.bias False
layer2.0.conv1.weight False
layer2.0.bn1.weight False
layer2.0.bn1.bias False
layer2.0.conv2.weight False
layer2.0.bn2.weight False
layer2.0.bn2.bias False
layer2.0.downsample.0.weight False
layer2.0.downsample.1.weight False
layer2.0.downsample.1.bias False
layer2.1.conv1.weight False
layer2.1.bn1.weight False
layer2.1.bn1.bias False
layer2.1.conv2.weight False
layer2.1.bn2.weight False
layer2.1.bn2.bias False
layer3.0.conv1.weight False
layer3.0.bn1.weight False
layer3.0.bn1.bias False
layer3.0.conv2.weight False
layer3.0.bn2.weight False
layer3.0.bn2.bias False
layer3.0.downsample.0.weight False
layer3.0.downsample.1.weight Fa

All the parameters of the model are frozen.

We will unfreeze only the final fc layer.

We usually freeze the first layers and fine-tune the later ones.

In [28]:
for param in resnet.fc.parameters():
    param.requires_grad = True

print_params(resnet)
print("Number of trainable parameters:", count_parameters(resnet))

conv1.weight False
bn1.weight False
bn1.bias False
layer1.0.conv1.weight False
layer1.0.bn1.weight False
layer1.0.bn1.bias False
layer1.0.conv2.weight False
layer1.0.bn2.weight False
layer1.0.bn2.bias False
layer1.1.conv1.weight False
layer1.1.bn1.weight False
layer1.1.bn1.bias False
layer1.1.conv2.weight False
layer1.1.bn2.weight False
layer1.1.bn2.bias False
layer2.0.conv1.weight False
layer2.0.bn1.weight False
layer2.0.bn1.bias False
layer2.0.conv2.weight False
layer2.0.bn2.weight False
layer2.0.bn2.bias False
layer2.0.downsample.0.weight False
layer2.0.downsample.1.weight False
layer2.0.downsample.1.bias False
layer2.1.conv1.weight False
layer2.1.bn1.weight False
layer2.1.bn1.bias False
layer2.1.conv2.weight False
layer2.1.bn2.weight False
layer2.1.bn2.bias False
layer3.0.conv1.weight False
layer3.0.bn1.weight False
layer3.0.bn1.bias False
layer3.0.conv2.weight False
layer3.0.bn2.weight False
layer3.0.bn2.bias False
layer3.0.downsample.0.weight False
layer3.0.downsample.1.weight Fa

Notice that we only have 5130 trainable parameters out of 11M.
Let's run the training and see if the model learns anything.

In [29]:
# You may have to play with the hyperparameters
run_training(model=resnet, num_epochs=20, learning_rate=0.0003)

[INFO] Using GPU: Tesla T4

100%|██████████| 40/40 [00:02<00:00, 13.43it/s, Epoch=0, Valid_Acc=38.7, Valid_Loss=1.89]
Validation Accuracy Improved (0.0 ---> 38.66)

100%|██████████| 40/40 [00:02<00:00, 13.46it/s, Epoch=1, Valid_Acc=42.9, Valid_Loss=1.74]
Validation Accuracy Improved (38.66 ---> 42.87)

100%|██████████| 40/40 [00:02<00:00, 13.37it/s, Epoch=2, Valid_Acc=43.7, Valid_Loss=1.69]
Validation Accuracy Improved (42.87 ---> 43.71)

100%|██████████| 40/40 [00:03<00:00, 12.87it/s, Epoch=3, Valid_Acc=45.1, Valid_Loss=1.64]
Validation Accuracy Improved (43.71 ---> 45.13)

100%|██████████| 40/40 [00:03<00:00, 13.17it/s, Epoch=4, Valid_Acc=45.5, Valid_Loss=1.62]
Validation Accuracy Improved (45.13 ---> 45.53)

100%|██████████| 40/40 [00:03<00:00, 12.02it/s, Epoch=5, Valid_Acc=46.2, Valid_Loss=1.61]
Validation Accuracy Improved (45.53 ---> 46.21)

100%|██████████| 40/40 [00:03<00:00, 13.22it/s, Epoch=6, Valid_Acc=45.8, Valid_Loss=1.62]

100%|██████████| 40/40 [00:03<00:00, 11.40it/s, E

**Exercise 1** - Unfreeze all the BatchNorm (bn) weights and biases and re-train the network. See what you observe.

- Play with freezing and unfreezing certain parts of the network. See if you can obtain good results without having to re-train too many parameters.

**Exercise 2** - Try to obtain the highest accuracy possible on CIFAR-10.
- Use any model you want from any of the presented libraries
- Use image augmentations in training and try to find optimal hyperparameters
- Models trained on ImageNet were trained with 224x224 images. What happens if you resize the CIFAR-10 32x32 images to the 224x224 size? (Training will take a very long time if you do this, but try it for 1-2 epochs)